In [1]:
import os
import json
import gzip
import io
import glob
import pickle
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from IPython.display import clear_output

In [2]:
def loadProgress():
    if(os.path.exists('dates_dict.pkl')):
        pkl_file = open('dates_dict.pkl', 'rb')
        date_dict = pickle.load(pkl_file)
        pkl_file.close()


    else: raise Exception('dates_dict.pkl file not found')

    if(os.path.exists('analysis_progress.pkl')):
        pkl_file1 = open('analysis_progress.pkl', 'rb')
        analysis_progress = pickle.load(pkl_file1)
        pkl_file1.close()  

    else:  analysis_progress = {}

    for date in date_dict:
        if date not in analysis_progress:
            analysis_progress.update( {date : "not_analyzed"} )
    return date_dict, analysis_progress


def sentimentAnalysis(tweets, date):
    i=1
    tweets_len = len(tweets)
    for tweet in tweets:
        tweet_dict = tweets[tweet]
        tweet_text = tweet_dict['tweet_text']
        analyzer = SentimentIntensityAnalyzer()
        vs = analyzer.polarity_scores(tweet_text)        
        tweets[tweet]['sentiment'] = vs['compound'] 

        if(i%100==0):
            print("Analysing "+str(i)+ "/"+str(tweets_len)+" tweets from "+str(date), end="\r")
        i +=1
    print("Analysing "+str(i)+ "/"+str(tweets_len)+" tweets from "+str(date))
    return tweets


In [ ]:
date_dict, analysis_progress = loadProgress()
        
tweets = {}
        
for date in date_dict:      
    if(date_dict[date]=="processed" and analysis_progress[date] == "not_analyzed"):

    #         loading the tweets
        start_time = time.time()
        tweet_pkl = open('Processed Tweets/tweets_' +date+'.pkl', 'rb')
        tweets = pickle.load(tweet_pkl)
        tweet_pkl.close()       

    #         calling the sentiment analysis function
        tweets = sentimentAnalysis(tweets, date)
    #         saving the results
        analysis_progress[date] = "analyzed"
        output = open('analysis_progress.pkl', 'wb')
        pickle.dump( analysis_progress, output)
        output.close()
        
        output = open('Processed Tweets/tweets_' +date+'.pkl', 'wb')
        print()
        pickle.dump( tweets, output)
        output.close()

        print('Analyzed '+ str(len(tweets)) + ' tweets from ' + str(date) + ' in ' + str(time.time()-start_time)+ 's')


Analysing 2037900/2037990 tweets from 2020-03-09

Analyzed 2037990 tweets from 2020-03-09 in 16173.662991523743s
